In [1]:
import os
import math

In [2]:
datasetPath = os.path.join("data","checked")
booklets = ["pandah_booklet1_lableled","pandah_booklet13_lableled","pandah_booklet17_lableled","tamimi_booklet0_lableled","tamimi_booklet3_lableled"]

In [ ]:
with open(os.path.join(datasetPath,booklets[0],"train.txt")) as t:
    imgsPath = t.readlines()

In [ ]:
# you only can use this line of code to sperate the file extension (jpg/JPEG) from the name of the file if the folder and files don't have a "." in their name
imgsTxtPath = list(map(lambda a: os.path.join(a.split(".")[0]+".txt") ,imgsPath))

In [8]:
#firt we want to order the YOLO boxes as

# offer_box/ class = 0 --parent
# and then any data box class that belongs to the above offer box --child

# to be able to attach the smaller boxes to its parent offer_box
# data box classes are (1,2,3,4,5) ~ (old_costs,new_costs,offers_name_arabic,offers_name_english,quantity) respectively

#example
# 0 0.137994 0.772300 0.241435 0.138200 -- parent
# 3 0.137116 0.812200 0.173499 0.021400 -- child
# 2 0.183968 0.787000 0.084480 0.026000 -- child
# 4 0.139531 0.832200 0.171596 0.016400 -- child
# 1 0.229868 0.768250 0.044802 0.010500 -- child
# 5 0.236384 0.787550 0.039678 0.027300 -- child
# 0 0.382357 0.921350 0.232650 0.135900 -- parent
# 3 0.379502 0.958150 0.223426 0.022700 -- child
# 4 0.379063 0.980000 0.198536 0.017600 -- child
# 1 0.471962 0.911750 0.046120 0.011300 -- child
# 5 0.478404 0.930200 0.039678 0.025800 -- child
# 2 0.426501 0.930100 0.084919 0.025400 -- child
# etc...

for booklet in booklets:
    
    #get the imgs paths and then turn the extension from (.jpg/.JPEG) to .txt
    with open(os.path.join(datasetPath,booklet,"train.txt")) as t:
         imgsPath = t.readlines()
    imgsTxtPath = list(map(lambda a: os.path.join(a.split(".")[0]+".txt") ,imgsPath))

    #take an booklet image/page at a time
    for imgtxtPath in imgsTxtPath:
        with open(os.path.join(datasetPath,booklet,imgtxtPath)) as t:
            boxes = t.readlines()

        #remove the \n in the last of every line
        boxes =list(map(lambda a: a[:-1],boxes))

        # convert to two dim array
        boxes =list(map(lambda a: a.split(" "),boxes))

        #seprate offer boxes with id "0" from the rest of the boxes
        offer_boxes= []
        data_boxes=[]
        for box in boxes:
            if(box[0]=="0"):
                offer_boxes.append(box)
            else:
                data_boxes.append(box)

        with open(os.path.join(datasetPath,booklet,imgtxtPath),"w"):
            pass
        # find what data boxes belong to what offer boxes by attaching data boxes that have their center point inside the offer box 
        # the data boxes that don't belong to any offer box will not be processed as they might be mislabeled
        for offer_box in offer_boxes:
            with open(os.path.join(datasetPath,booklet,imgtxtPath),"a") as t:
                t.writelines(str.join(" ",offer_box))
                t.writelines("\n")
            offerBox_x_center= float(offer_box[1])
            offerBox_y_center= float(offer_box[2])
            offerBox_width= float(offer_box[3])
            offerBox_height= float(offer_box[4])

            lowest_x= offerBox_x_center-offerBox_width/2
            highest_x= offerBox_x_center+offerBox_width/2
            lowest_y=offerBox_y_center-offerBox_height/2
            highest_y=offerBox_y_center+offerBox_height/2
            current_offer_data=[]
            for data_box in data_boxes:
                dataBox_x_center = float(data_box[1])
                dataBox_y_center = float(data_box[2])

                if(dataBox_x_center >= lowest_x and dataBox_x_center <= highest_x):
                    if(dataBox_y_center>=lowest_y and dataBox_y_center <= highest_y):
                        with open(os.path.join(datasetPath,booklet,imgtxtPath),"a") as t:
                            t.writelines(str.join(" ",data_box))
                            t.writelines("\n")


In [24]:
from ntpath import join
from re import L
import cv2 as cv


offerDatasetPath=os.path.join("data","offers only dataset")

#now we create the offers dataset by cutting each offer from thier page and save them along with the the offer's data_boxs dim in YOLO format 

for booklet in booklets:
    print(booklets)
    with open(os.path.join(datasetPath,booklet,"train.txt")) as t:
         imgsPath = t.readlines()
    pagen=0
    for imgpath in imgsPath :
        print(imgpath)
        img = cv.imread(str(os.path.join(datasetPath,booklet,imgpath[:-1])))
        pagen=pagen+1
        imgHeight, imgWidth, _ = img.shape

        with open(os.path.join(datasetPath,booklet,imgpath.split(".")[0]+".txt")) as t:
            boxes = t.readlines()
        offern=0
        for box in boxes:
            print(box)
            box= box.split(" ")
            offerBox_x_center = float(box[1])
            offerBox_y_center = float(box[2])
            offerBox_width = float(box[3])
            offerBox_height = float(box[4])

            lowest_x = offerBox_x_center-offerBox_width/2
            highest_x = offerBox_x_center+offerBox_width/2
            lowest_y = offerBox_y_center-offerBox_height/2
            highest_y = offerBox_y_center+offerBox_height/2
            print(f"lowest relative:{lowest_x}\ninpixles:{lowest_x*imgWidth}")
            #convent to pixels 
            # the -1 becuase img.shape == index + 1
            # will never be  nigative because image height and width are always greater than 0 
            lowest_x = int(math.ceil(lowest_x*imgWidth)) -1
            highest_x = int(math.ceil(highest_x*imgWidth)) -1
            lowest_y = int(math.ceil(lowest_y*imgHeight)) -1
            highest_y = int(math.ceil(highest_y*imgHeight)) -1

            if(box[0]=="0"):
                #cut the image then save it 
                offern=offern+1
                parent_l_x = lowest_x
                parent_h_x = highest_x
                parent_l_y = lowest_y
                parent_h_y = highest_y

                offer_box_img = img[parent_l_y:parent_h_y,parent_l_x:parent_h_x]
                print(str(offer_box_img.shape))
                offerImgHeight, offerImgWidth, _ = offer_box_img.shape
                
                temp=imgpath.split("/")[1][:-1]
                print(os.path.join(offerDatasetPath,booklet,f"s{booklet}_p{pagen}_o{offern}-{temp}"))
                cv.imwrite(os.path.join(offerDatasetPath,f"s{booklet}_p{pagen}_o{offern}-{temp}"),offer_box_img)
                print(f"image been cutten with:\nlowest_x:{lowest_x}\nhighest_x:{highest_x}\nlowest_y:{lowest_y}\nhighest_y:{highest_y}\noffern{offern}")

            else:
                # convert the dim of the data_box to be relative to the new offer image
                print(f"dataBox before being relative:\nlowest_x:{lowest_x}\nhighest_x:{highest_x}\nlowest_y:{lowest_y}\nhighest_y:{highest_y}\noffern{offern}")

                lowest_x = lowest_x-parent_l_x
                highest_x= highest_x-parent_l_x
                lowest_y= lowest_y-parent_l_y
                highest_y= highest_y-parent_l_y
                print(f"dataBox after being relative:\nlowest_x:{lowest_x}\nhighest_x:{highest_x}\nlowest_y:{lowest_y}\nhighest_y:{highest_y}\noffern{offern}")
                # then save it in the txt file of the previos offer box
                dataBox_width= (highest_x-lowest_x)/offerImgWidth
                dataBox_height= (highest_y-lowest_y)/offerImgHeight
                dataBox_x_center= (dataBox_width/2+lowest_x/offerImgWidth)
                dataBox_y_center= (dataBox_height/2+lowest_y/offerImgHeight)
                print(f"dataBox after being YOLO:\ndataBox_width:{dataBox_width}\ndataBox_height:{dataBox_height}\ndataBox_x_center:{dataBox_x_center}\ndataBox_y_center:{dataBox_y_center}\noffern{offern}")

                with open(os.path.join(offerDatasetPath,f"s{booklet}_p{pagen}_o{offern}-"+imgpath.split("/")[1].split(".")[0]+".txt"),"a") as t:
                    t.write(f"{int(box[0])-1} {dataBox_x_center} {dataBox_y_center} {dataBox_width} {dataBox_height}")
                    t.write("\n")

['pandah_booklet1_lableled', 'pandah_booklet13_lableled', 'pandah_booklet17_lableled', 'tamimi_booklet0_lableled', 'tamimi_booklet3_lableled']
obj_train_data/1421546695.jpg

0 0.499154 0.393100 0.352995 0.257400

lowest relative:0.3226565
inpixles:247.800192
(257, 271, 3)
data\offers only dataset\pandah_booklet1_lableled\spandah_booklet1_lableled_p1_o1-1421546695.jpg
image been cutten with:
lowest_x:247
highest_x:518
lowest_y:264
highest_y:521
offern1
2 0.401172 0.491900 0.133854 0.044800

lowest relative:0.33424499999999996
inpixles:256.70016
dataBox before being relative:
lowest_x:256
highest_x:359
lowest_y:469
highest_y:514
offern1
dataBox after being relative:
lowest_x:9
highest_x:112
lowest_y:205
highest_y:250
offern1
dataBox after being YOLO:
dataBox_width:0.3800738007380074
dataBox_height:0.17509727626459143
dataBox_x_center:0.22324723247232472
dataBox_y_center:0.8852140077821012
offern1
5 0.498112 0.497650 0.058203 0.038100

lowest relative:0.4690105
inpixles:360.200064
dataBox

In [ ]:
import cv2 as cv
img = cv.imread(
    datasetPath+"\\"+booklet+"\\"+"data\\obj_train_data\\1381547765.jpg")
dh, dw, _ = img.shape

img.shape
print()
